## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pyapon,MM,2021-07-24 13:19:05,16.2833,95.6833,80.04,88,100,9.51,heavy intensity rain
1,1,Victoria,HK,2021-07-24 13:19:05,22.2855,114.1577,90.21,81,78,1.01,broken clouds
2,2,Ancud,CL,2021-07-24 13:19:06,-41.8697,-73.8203,35.13,82,100,5.26,overcast clouds
3,3,Punta Arenas,CL,2021-07-24 13:19:06,-53.1500,-70.9167,37.42,87,40,13.80,scattered clouds
4,4,Nikolskoye,RU,2021-07-24 13:19:06,59.7035,30.7861,71.40,32,9,7.38,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Ribeira Grande,PT,2021-07-24 13:19:07,38.5167,-28.7000,79.20,69,40,19.57,scattered clouds
11,11,Arraial Do Cabo,BR,2021-07-24 13:19:08,-22.9661,-42.0278,75.16,60,20,16.11,few clouds
12,12,Altay,CN,2021-07-24 13:19:09,47.8667,88.1167,79.57,35,1,11.18,clear sky
14,14,Atuona,PF,2021-07-24 13:19:09,-9.8000,-139.0333,77.68,70,97,15.14,overcast clouds
29,29,Kapaa,US,2021-07-24 13:19:14,22.0752,-159.3190,79.12,82,90,3.00,light rain
32,32,Konya,TR,2021-07-24 13:19:15,38.1667,32.5000,76.48,25,2,11.72,clear sky
38,38,Marsh Harbour,BS,2021-07-24 13:19:17,26.5412,-77.0636,79.03,83,100,11.27,overcast clouds
39,39,Padang,ID,2021-07-24 13:19:17,-0.9492,100.3543,78.85,85,90,3.29,overcast clouds
43,43,Galle,LK,2021-07-24 13:19:18,6.0367,80.2170,79.39,86,76,10.80,broken clouds
50,50,Axim,GH,2021-07-24 13:19:20,4.8699,-2.2405,79.99,65,90,9.86,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                119
City                   119
Country                118
Date                   119
Lat                    119
Lng                    119
Max Temp               119
Humidity               119
Cloudiness             119
Wind Speed             119
Current Description    119
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Ribeira Grande,PT,79.20,scattered clouds,38.5167,-28.7000,
11,Arraial Do Cabo,BR,75.16,few clouds,-22.9661,-42.0278,
12,Altay,CN,79.57,clear sky,47.8667,88.1167,
14,Atuona,PF,77.68,overcast clouds,-9.8000,-139.0333,
29,Kapaa,US,79.12,light rain,22.0752,-159.3190,
32,Konya,TR,76.48,clear sky,38.1667,32.5000,
38,Marsh Harbour,BS,79.03,overcast clouds,26.5412,-77.0636,
39,Padang,ID,78.85,overcast clouds,-0.9492,100.3543,
43,Galle,LK,79.39,broken clouds,6.0367,80.2170,
50,Axim,GH,79.99,overcast clouds,4.8699,-2.2405,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    

   # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        print(index)
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")

5
11
12
14
29
32
38
39
43
50
57
66
67
Hotel not found... skipping.
74
88
91
95
113
116
119
124
146
154
161
170
172
186
187
195
196
199
205
206
213
214
217
221
227
228
234
236
239
257
259
260
267
273
281
290
298
299
300
302
304
309
319
337
341
344
353
358
367
369
376
378
379
Hotel not found... skipping.
387
396
397
403
406
408
418
419
424
426
432
442
Hotel not found... skipping.
443
444
Hotel not found... skipping.
447
450
455
459
474
485
496
505
517
523
531
533
541
547
548
549
Hotel not found... skipping.
556
558
569
Hotel not found... skipping.
573
576
Hotel not found... skipping.
581
584
585
592
600
602
605
619
629
633
634
Hotel not found... skipping.
635
649
652
654
657
667


In [26]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 5 to 667
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 118 non-null    object 
 1   Country              118 non-null    object 
 2   Max Temp             118 non-null    float64
 3   Current Description  118 non-null    object 
 4   Lat                  118 non-null    float64
 5   Lng                  118 non-null    float64
 6   Hotel Name           118 non-null    object 
dtypes: float64(3), object(4)
memory usage: 11.4+ KB


In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 5 to 667
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 118 non-null    object 
 1   Country              118 non-null    object 
 2   Max Temp             118 non-null    float64
 3   Current Description  118 non-null    object 
 4   Lat                  118 non-null    float64
 5   Lng                  118 non-null    float64
 6   Hotel Name           118 non-null    object 
dtypes: float64(3), object(4)
memory usage: 7.4+ KB


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))